# Coding up your own model

With the new scheme, the pipeline can interpret any model passed to it, no matter the number of parameters or the ordering of the ingredients, provided that the configuration file is consistent with the model structure.

Several functional forms have been implemented in `kids_ggl_pipeline/helpers/functions.py`, including a double power law and a power law with both mass and redshift,

$$
\log m_\star = A + B\log\left(\frac{M_\mathrm{h}}{M_0}\right) + C\left(\frac{1+z}{1+z_0}\right)\,.
$$

This function, as well as any other custom function, must include the halo mass as the first argument; all other arguments must be defined in the configuration file -- including the redshift, in this case. This would therefore require the central MOR section in the configuration file to look like

```
[hod/centrals/mor]
name           powerlaw_mz
z              array          0.188,0.195
logM0          fixed          14
z0             fixed          0.2
A              uniform        10      16    12
B              uniform        0       5     1
C              uniform        -1      1     0
```

*The only condition is that the order of sections **must follow the order defined in the coded model**.*

For the time being, additional scaling relations must be included in the file `kids_ggl_pipeline/halomodel/hod/relations.py`, and custom distributions for the scatter about this relation must be included in the file `kids_ggl_pipeline/halomodel/hod/scatter.py`. (This has the undesirable effect that this file, common to all users, might get clogged with trial-and-error attempts by various users, so please try to only push files with working models (and if you have a new working model, please push it!). We will implement the ability to input user-provided files in the future.)

The user may use the generic functions and distributions located in `kids_ggl_pipeline/helpers/functions.py` and `kids_ggl_pipeline/helpers/distributions.py`. For instance, the Duffy et al. (2008) mass-concentration relation is implemented in `kids_ggl_pipeline/halomodel/concentration.py` as:

In [ ]:
from kids_ggl_pipeline.helpers.functions import powerlaw_mz

def duffy08_crit(M, z, f, h=1):
    return f * powerlaw_mz(
        M, z[:,newaxis], 12.301-log10(h), 0, 0.8267, -0.091, -0.44,
        return_log=False)

the `return_log` keyword argument allows the function to be automatically interpreted as being requested in linear or log space depending on how the observable was defined, with the help of the `logfunc` decorator found in `kids_ggl_pipeline/helpers/decorators.py`. For this to work correctly, the function (in this case `powerlaw_mz`) **must be defined in log-space**:

In [ ]:
@logfunc
def powerlaw_mz(M, z, logM0, z0, a, b, c, return_log=True):
    return a + b*(log10(M)-logM0) + c*log10((1+z)/(1+z0))